# 变量:创建、初始化、保存和加载

##### 
当训练模型时，用变量来存储和更新参数。变量包含张量 (Tensor)存放于内存的缓存区。建模时它们需要被明确地初始化，模型训练后它们必须被存储到磁盘。这些变量的值可在之后模型训练和分析是被加载。

本文档描述以下两个TensorFlow类。点击以下链接可查看完整的API文档：

- tf.Variable 类
- tf.train.Saver 类

## 创建

当创建一个变量时，将张量作为初始值传入构造函数Variable()。Tensorflow提供操作符初始化张量，初始值是常量。
*注意，操作符需要指定张量的shape,那个形状自动成为变量的shape*

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
# Create two variables.
weights=tf.Variable(tf.random_normal([784,200],stddev=0.35),name="weights")
biases=tf.Variable(tf.zeros([200]),name="biases")

调用tf.Variable()添加OP到Graph:
- 一个variable操作存放变量的值
- 一个初始化op将变量设置为初始值。事实上是tf.assign操作
- 初始值的操作，例如示例的bias变量和zeros操作也被加入graph
tf.Varibale返回python的tf.Variable类的实例


## 初始化


变量的初始化必须在模型的其它操作运行之前先明确地完成。最简单的方法就是添加一个给所有变量初始化的操作，并在使用模型之前首先运行那个操作。

In [3]:
#add an op to initialize the variable
init_op=tf.global_variables_initializer()
#later,when launching the model
with tf.Session() as sess:
    #Run the init op
    sess.run(init_op)
    #...

### 由另一个变量初始化

你有时候会需要用另一个变量的初始化值给当前变量初始化。由于tf.global_variables_initializer()是并行地初始化所有变量，所以在有这种需求的情况下需要小心。

用其它变量的值初始化一个新的变量时，使用其它变量的initialized_value()属性。你可以直接把已初始化的值作为新变量的初始值，或者把它当做tensor计算得到一个值赋予新变量。

In [4]:
#create' another variable with the same value as 'weights'
#w2=tf.Variable(weights.initialized_value(),name="w2")

### 保存和加载


最简单的保存和恢复模型的方法是使用tf.train.Saver对象。构造器给graph的所有变量，或是定义在列表里的变量，添加save和restoreops。saver对象提供了方法来运行这些ops，定义检查点文件的读写路径。


In [5]:
#add ops to save and restore all the variables
saver=tf.train.Saver()

#Later,launch the model,initialize variables,do some work, save variables to disk
with tf.Session() as sess:
    sess.run(init_op)
    save_path=saver.save(sess,"./models/model.ckpt")

### 恢复变量
用同一个Saver对象来恢复变量。注意，当你从文件中恢复变量时，不需要事先对它们做初始化。

In [6]:
with tf.Session() as sess:
    saver.restore(sess,"./models/model.ckpt")

INFO:tensorflow:Restoring parameters from ./models/model.ckpt
